In [8]:
import numpy as np
import torch
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, f1_score
import time

# Qiskit imports
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.quantum_info import SparsePauliOp
from qiskit_aer.noise import NoiseModel
from qiskit_aer.noise.errors import depolarizing_error, amplitude_damping_error, phase_damping_error, pauli_error
from qiskit.quantum_info import Kraus


In [9]:

# Configuration
n_qubits = 2
noise_prob = 0.3   # Change this to set noise probability (0.0 = noiseless)
noise_type = "AmplitudeDamping"  # Options: "Depolarizing", "AmplitudeDamping", "PhaseDamping", "BitFlip", "PhaseFlip"

# Data preparation
iris = datasets.load_iris()
X = iris.data
y = iris.target

# Binary classification (Setosa vs Versicolor)
mask = y < 2
X = X[mask]
y = y[mask]

# Standardize
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Use only first 2 features
X = X[:, :2]
y = y.astype(float)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Torch tensors in float64
X_train = torch.tensor(X_train, dtype=torch.float64)
y_train = torch.tensor(y_train, dtype=torch.float64)
X_test = torch.tensor(X_test, dtype=torch.float64)
y_test = torch.tensor(y_test, dtype=torch.float64)

def create_noise_model(noise_type, noise_prob):
    """Create Qiskit noise model"""
    if noise_prob == 0.0:
        return None
    
    noise_model = NoiseModel()
    
    if noise_type == "Depolarizing":
        error = depolarizing_error(noise_prob, 1)
    elif noise_type == "AmplitudeDamping":
        error = amplitude_damping_error(noise_prob)
    elif noise_type == "PhaseDamping":
        error = phase_damping_error(noise_prob)
    elif noise_type == "BitFlip":
        # Create bit flip error using pauli_error
        error = pauli_error([('X', noise_prob), ('I', 1 - noise_prob)])
    elif noise_type == "PhaseFlip":
        # Create phase flip error using pauli_error
        error = pauli_error([('Z', noise_prob), ('I', 1 - noise_prob)])
    else:
        return None
    
    # Apply noise to all single-qubit gates
    noise_model.add_all_qubit_quantum_error(error, ['rx', 'ry', 'rz'])
    # Apply noise to CNOT gates (using 2-qubit depolarizing for entangling gates)
    if noise_type == "Depolarizing":
        cnot_error = depolarizing_error(noise_prob, 2)
        noise_model.add_all_qubit_quantum_error(cnot_error, ['cx'])
    
    return noise_model

def feature_encoding(circuit, x):
    """Encode features using RX rotations"""
    for i in range(n_qubits):
        circuit.rx(float(x[i]), i)

def shallow_HEA(circuit, weights):
    """Shallow Hardware Efficient Ansatz"""
    # Convert weights to numpy if it's a tensor
    if isinstance(weights, torch.Tensor):
        weights = weights.detach().cpu().numpy()
    
    # Parameterized rotations
    for i in range(n_qubits):
        circuit.rx(float(weights[i, 0]), i)
        circuit.rz(float(weights[i, 1]), i)
    
    # Entangling gate
    circuit.cx(0, 1)

def create_circuit(weights, x):
    """Create the complete quantum circuit"""
    circuit = QuantumCircuit(n_qubits)
    
    # Feature encoding
    feature_encoding(circuit, x)
    
    # Shallow HEA
    shallow_HEA(circuit, weights)
    
    return circuit

def get_expectation_value(circuit, noise_model=None, shots=8192):
    """Calculate expectation value of Pauli-Z on qubit 0"""
    # Create the observable (Pauli-Z on qubit 0)
    observable = SparsePauliOp(['Z' + 'I' * (n_qubits - 1)], coeffs=[1.0])
    
    # Use AerSimulator
    if noise_model is not None:
        simulator = AerSimulator(noise_model=noise_model)
    else:
        simulator = AerSimulator(method='statevector')
    
    # For noisy simulation, use sampling
    if noise_model is not None:
        # Add measurements to circuit
        circuit_copy = circuit.copy()
        circuit_copy.measure_all()
        
        # Transpile and run
        transpiled = transpile(circuit_copy, simulator)
        job = simulator.run(transpiled, shots=shots)
        result = job.result()
        counts = result.get_counts()
        
        # Calculate expectation value from measurement statistics
        expectation = 0.0
        total_shots = sum(counts.values())
        
        for bitstring, count in counts.items():
            # For Pauli-Z on qubit 0, we look at the rightmost bit (qubit 0)
            z_eigenvalue = 1 if bitstring[-1] == '0' else -1
            expectation += z_eigenvalue * count / total_shots
            
    else:
        # For noiseless simulation, use statevector
        from qiskit.quantum_info import Statevector
        transpiled = transpile(circuit, simulator)
        job = simulator.run(transpiled)
        result = job.result()
        statevector = result.get_statevector()
        expectation = statevector.expectation_value(observable).real
    
    return expectation

class QiskitQuantumLayer:
    """Quantum layer using Qiskit"""
    def __init__(self, noise_model):
        self.noise_model = noise_model
    
    def forward(self, weights, x):
        """Forward pass through quantum circuit"""
        # Convert tensor inputs to numpy
        if isinstance(x, torch.Tensor):
            x = x.detach().cpu().numpy()
        
        # Create and execute circuit
        circuit = create_circuit(weights, x)
        expectation = get_expectation_value(circuit, self.noise_model)
        
        return expectation

class QuantumClassifier(torch.nn.Module):
    def __init__(self, noise_model):
        super().__init__()
        self.weights = torch.nn.Parameter(0.01 * torch.randn(n_qubits, 2, dtype=torch.float64))
        self.quantum_layer = QiskitQuantumLayer(noise_model)
        
    def forward(self, x):
        outputs = []
        for xi in x:
            # Get expectation value from quantum circuit
            expectation = self.quantum_layer.forward(self.weights, xi)
            outputs.append(expectation)
        
        # Convert to tensor and map from [-1,1] to [0,1]
        outputs = torch.tensor(outputs, dtype=torch.float64)
        return (outputs.reshape(-1, 1) * 0.5 + 0.5)

# Custom autograd function for quantum layer
class QuantumFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, weights, x, quantum_layer):
        ctx.quantum_layer = quantum_layer
        ctx.save_for_backward(weights, x)
        
        # Forward pass
        outputs = []
        for xi in x:
            expectation = quantum_layer.forward(weights, xi)
            outputs.append(expectation)
        
        return torch.tensor(outputs, dtype=torch.float64)
    
    @staticmethod
    def backward(ctx, grad_output):
        weights, x = ctx.saved_tensors
        quantum_layer = ctx.quantum_layer
        
        # Parameter-shift rule for gradients
        epsilon = np.pi / 2  # For parameter-shift rule
        grad_weights = torch.zeros_like(weights)
        
        for i in range(weights.shape[0]):
            for j in range(weights.shape[1]):
                grad_param = 0.0
                
                for k, xi in enumerate(x):
                    # Shift parameter forward
                    weights_plus = weights.clone()
                    weights_plus[i, j] += epsilon
                    exp_plus = quantum_layer.forward(weights_plus, xi)
                    
                    # Shift parameter backward  
                    weights_minus = weights.clone()
                    weights_minus[i, j] -= epsilon
                    exp_minus = quantum_layer.forward(weights_minus, xi)
                    
                    # Parameter-shift gradient
                    grad_param += grad_output[k] * (exp_plus - exp_minus) / 2.0
                
                grad_weights[i, j] = grad_param
        
        return grad_weights, None, None

class QuantumClassifierWithGrad(torch.nn.Module):
    def __init__(self, noise_model):
        super().__init__()
        self.weights = torch.nn.Parameter(0.01 * torch.randn(n_qubits, 2, dtype=torch.float64))
        self.quantum_layer = QiskitQuantumLayer(noise_model)
        
    def forward(self, x):
        # Use custom autograd function
        quantum_outputs = QuantumFunction.apply(self.weights, x, self.quantum_layer)
        # Map from [-1,1] to [0,1]
        return (quantum_outputs.reshape(-1, 1) * 0.5 + 0.5)


In [10]:
# Create noise model
noise_model = create_noise_model(noise_type, noise_prob)

# Create model (use simpler version for faster training)
model = QuantumClassifier(noise_model)

# Training setup
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
loss_fn = torch.nn.BCELoss()

print("Starting training...")
print(f"Noise type: {noise_type}, Noise probability: {noise_prob}")

# Training loop
epochs = 30
for epoch in range(epochs):
    optimizer.zero_grad()
    
    # Forward pass
    y_pred = model(X_train)
    loss = loss_fn(y_pred, y_train.reshape(-1, 1))
    
    # Backward pass with finite differences (since parameter-shift is complex)
    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % 5 == 0:
        with torch.no_grad():
            y_pred_eval = model(X_train)
            acc = ((y_pred_eval.detach().numpy() > 0.5) == y_train.numpy().reshape(-1, 1)).mean()
            print(f"Epoch {epoch+1:2d} | Loss: {loss.item():.4f} | Train Acc: {acc*100:.2f}%")

print("\nTraining completed!")

Starting training...
Noise type: AmplitudeDamping, Noise probability: 0.3


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [11]:
# Evaluation
print("Evaluating on test set...")
with torch.no_grad():
    start_time = time.perf_counter()
    
    preds = model(X_test).detach().numpy()
    preds_binary = (preds > 0.5).astype(int)
    
    end_time = time.perf_counter()
    inference_time_per_sample = (end_time - start_time) / len(X_test)
    
    y_true_np = y_test.detach().numpy().flatten()
    
    # Metrics
    mse = mean_squared_error(y_true_np, preds.flatten())
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true_np, preds.flatten())
    
    # F1 score
    y_true_int = y_test.detach().numpy().astype(int)
    f1 = f1_score(y_true_int, preds_binary.flatten())

print(f"\n=== SHEA-P Qiskit Evaluation ===")
print(f"Noise: {noise_type}")
print(f"Noise probability: {noise_prob}")
print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"Inference time per sample: {inference_time_per_sample:.6f} sec")
print(f"No. of qubits: {n_qubits}")


Evaluating on test set...

=== SHEA-P Qiskit Evaluation ===
Noise: AmplitudeDamping
Noise probability: 0.3
MSE: 0.5377
RMSE: 0.7333
MAE: 0.6060
F1-score: 0.5714
Inference time per sample: 0.026233 sec
No. of qubits: 2
